# My Content - Recommender System

## Téléchargement et Prétraitement des Données

Dans ce notebook, nous allons télécharger les fichiers de données et les prétraiter pour le système de recommandation.

In [ ]:
# Importation des bibliothèques nécessaires
import os
import requests
import zipfile
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from flask import Flask, request, jsonify, render_template
import joblib

# Importation de TensorFlow
import tensorflow as tf

# Importation de MLflow
import mlflow
import mlflow.sklearn

# Importation des bibliothèques de scikit-learn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


## Téléchargement des fichiers de données

Nous allons télécharger les fichiers de données depuis les liens fournis et les enregistrer dans le répertoire `../data`.

In [ ]:
# Fonction pour télécharger un fichier
def download_file(url, dest_path):
    if not os.path.exists(dest_path):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Vérifie si la requête a réussi
            with open(dest_path, 'wb') as file:
                file.write(response.content)
            print(f'Téléchargement terminé : {dest_path}')
        except requests.exceptions.RequestException as e:
            print(f'Erreur lors du téléchargement : {e}')
    else:
        print(f'Le fichier {dest_path} existe déjà. Téléchargement non nécessaire.')

# URL du fichier zip contenant les données
data_url = 'https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+9+-+R%C3%A9alisez+une+application+mobile+de+recommandation+de+contenu/news-portal-user-interactions-by-globocom.zip'

# Chemin de destination pour le fichier zip
zip_file_path = '../data/news-portal-user-interactions-by-globocom.zip'

# Création du répertoire data s'il n'existe pas
if not os.path.exists('../data'):
    os.makedirs('../data', exist_ok=True)
    print('Répertoire ../data créé.')

# Téléchargement du fichier zip
download_file(data_url, zip_file_path)

# Décompression du fichier zip
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall('../data')
    print('Extraction terminée.')
except zipfile.BadZipFile:
    print('Erreur : Le fichier zip est corrompu.')

# Chemins des fichiers extraits
clicks_zip_path = '../data/clicks.zip'
clicks_sample_path = '../data/clicks_sample.csv'
articles_metadata_path = '../data/articles_metadata.csv'
articles_embeddings_path = '../data/articles_embeddings.pickle'

# Vérification des fichiers extraits
if all(os.path.exists(path) for path in [clicks_zip_path, clicks_sample_path, articles_metadata_path, articles_embeddings_path]):
    print(f'Fichiers extraits:\n- {clicks_zip_path}\n- {clicks_sample_path}\n- {articles_metadata_path}\n- {articles_embeddings_path}')
else:
    print('Erreur lors de l\'extraction des fichiers.')

# Décompression de clicks.zip
if os.path.exists(clicks_zip_path):
    try:
        with zipfile.ZipFile(clicks_zip_path, 'r') as zip_ref:
            zip_ref.extractall('../data')
        print('Extraction de clicks.zip terminée.')
    except zipfile.BadZipFile:
        print('Erreur : Le fichier clicks.zip est corrompu.')

# Chemin des fichiers extraits de clicks.zip
clicks_path = '../data/clicks.zip'

# Décompression de clicks.zip
clicks_extract_dir = '../data/clicks/'
if not os.path.exists(clicks_extract_dir):
    os.makedirs(clicks_extract_dir, exist_ok=True)
    try:
        with zipfile.ZipFile(clicks_zip_path, 'r') as zip_ref:
            zip_ref.extractall(clicks_extract_dir)
        print('Extraction de clicks.zip terminée.')
    except zipfile.BadZipFile:
        print('Erreur : Le fichier clicks.zip est corrompu.')
else:
    print(f'Le répertoire {clicks_extract_dir} existe déjà. Extraction non nécessaire.')



## Chargement et aperçu des données

Nous allons maintenant charger les fichiers CSV et afficher un aperçu des données.

In [ ]:
# Lecture des fichiers CSV extraits de clicks.zip
clicks_files = [os.path.join(clicks_extract_dir, f) for f in os.listdir(clicks_extract_dir) if f.endswith('.csv')]

# Lecture des autres fichiers
clicks_sample_df = pd.read_csv(clicks_sample_path)
articles_metadata_df = pd.read_csv(articles_metadata_path)

# Lecture du fichier pickle
with open(articles_embeddings_path, 'rb') as file:
    articles_embeddings = pickle.load(file)

# Aperçu des autres fichiers
print("Aperçu du fichier clicks_sample.csv:")
print(clicks_sample_df.head(), "\n")

print("Informations sur le fichier clicks_sample.csv:")
print(clicks_sample_df.info(), "\n")

print("Aperçu du fichier articles_metadata.csv:")
print(articles_metadata_df.head(), "\n")

print("Informations sur le fichier articles_metadata.csv:")
print(articles_metadata_df.info(), "\n")

print("Aperçu des embeddings d'articles:")
print(f"Nombre d'articles : {len(articles_embeddings)}")
print(f"Exemple d'embedding pour le premier article : {articles_embeddings[0]}")

clicks_sample.csv :
Contient des données similaires aux fichiers extraits de clicks.zip.
Aperçu des 5 premières lignes et informations sur les colonnes et types de données.
articles_metadata.csv :
Contient des métadonnées sur les articles, comme le nombre de mots et autres caractéristiques.
Aperçu des 5 premières lignes et informations sur les colonnes et types de données.
articles_embeddings.pickle :
Contient les embeddings des articles (représentations vectorielles).
Nombre total d'articles : 364047
Exemple d'embedding pour le premier article : [0.08109499, 0.0563579, 0.21711417, ...]

In [ ]:
# Distribution des clics par article dans clicks_sample.csv
clicks_sample_df['click_article_id'].value_counts().head(10).plot(kind='bar')
plt.title('Top 10 des articles les plus cliqués')
plt.xlabel('ID de l\'article')
plt.ylabel('Nombre de clics')
plt.show()

# Répartition des clics par environnement de clic dans clicks_sample.csv
clicks_sample_df['click_environment'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title('Répartition des clics par environnement de clic')
plt.ylabel('')
plt.show()

# Distribution du nombre de mots par article dans articles_metadata.csv
articles_metadata_df['words_count'].plot(kind='hist', bins=30)
plt.title('Distribution du nombre de mots par article')
plt.xlabel('Nombre de mots')
plt.ylabel('Nombre d\'articles')
plt.show()


## Préparation des données
Nous allons maintenant prétraiter les données pour le modèle de recommandation.

In [ ]:
# Analyse des données
analysis_results = {}
# Valeurs manquantes
analysis_results['clicks_sample_missing'] = clicks_sample_df.isnull().sum()
analysis_results['articles_metadata_missing'] = articles_metadata_df.isnull().sum()

# Doublons
analysis_results['clicks_sample_duplicates'] = clicks_sample_df.duplicated().sum()
analysis_results['articles_metadata_duplicates'] = articles_metadata_df.duplicated().sum()

analysis_results

# Développement du Modèle de Recommandation
- Filtrage collaboratif article-article basé sur les clics.
- Calcul de la similarité cosinus entre les articles.

preapration des data

In [ ]:
# Transformation des données
# Encodage des variables catégorielles
label_encoder = LabelEncoder()
clicks_sample_df['user_id'] = label_encoder.fit_transform(clicks_sample_df['user_id'])
clicks_sample_df['article_id'] = label_encoder.fit_transform(clicks_sample_df['article_id'])

# Normalisation ou standardisation si nécessaire (exemple ici non inclus)

# Features basées sur le texte
tfidf_vectorizer = TfidfVectorizer(max_features=100)
tfidf_matrix = tfidf_vectorizer.fit_transform(articles_metadata_df['article_content'])

# Construction des matrices utilisateur-article
user_article_matrix = clicks_sample_df.pivot(index='user_id', columns='article_id', values='interaction')
user_article_matrix.fillna(0, inplace=True)

# Sauvegarde des matrices pour une utilisation ultérieure
user_article_matrix.to_csv('user_article_matrix.csv')

# Aperçu des transformations
print("Aperçu de clicks_sample_df:")
print(clicks_sample_df.head(), "\n")
print("Aperçu de articles_metadata_df:")
print(articles_metadata_df.head(), "\n")
print("Aperçu de la matrice utilisateur-article:")
print(user_article_matrix.head(), "\n")

## Fonction de recommandation basée sur la similarité article-article

In [ ]:
# Préparation des données pour les modèles de recommandation
try:
    # Filtrage collaboratif article-article basé sur les clics
    article_user_matrix = clicks_sample_df.pivot_table(index='user_id', columns='click_article_id', values='session_id', aggfunc='count').fillna(0)
    
    # Vérifions la matrice utilisateur-article
    print("Aperçu de la matrice utilisateur-article:")
    print(article_user_matrix.head())

    # Convertir la matrice utilisateur-article en tenseur TensorFlow
    article_user_matrix_tensor = tf.convert_to_tensor(article_user_matrix.values, dtype=tf.float32)

    # Normaliser la matrice utilisateur-article
    normalized_article_user_matrix = tf.nn.l2_normalize(article_user_matrix_tensor, axis=0)

    # Calcul de la similarité cosinus entre les articles en utilisant TensorFlow
    article_similarity = tf.matmul(tf.transpose(normalized_article_user_matrix), normalized_article_user_matrix).numpy()
    
    # Vérifions la matrice de similarité
    print("Aperçu de la matrice de similarité:")
    print(article_similarity[:5, :5])  # Affichage d'un aperçu pour éviter trop de données

    # Fonction de recommandation basée sur la similarité article-article
    def recommend_articles_article_based(article_id, num_recommendations=5):
        article_idx = article_user_matrix.columns.get_loc(article_id)
        similar_articles = list(enumerate(article_similarity[article_idx]))
        similar_articles = sorted(similar_articles, key=lambda x: x[1], reverse=True)
        recommended_articles = [article_user_matrix.columns[i[0]] for i in similar_articles[1:num_recommendations+1]]
        return recommended_articles

    # Exemple de recommandation d'articles pour un article donné
    article_id_example = article_user_matrix.columns[0]
    print(f"Recommandations pour l'article {article_id_example} : {recommend_articles_article_based(article_id_example)}")
    
except Exception as e:
    print(f"Une erreur s'est produite: {e}")


## Recommandation basée sur les métadonnées des articles (TF-IDF et similarité cosinus)

In [ ]:
# Vérifiez si TensorFlow utilise le GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Chargement des données
articles_metadata_df = pd.read_csv('../data/articles_metadata.csv')

# Ajouter des colonnes fictives 'title' et 'content' pour la démonstration
articles_metadata_df['title'] = ["Example Title"] * len(articles_metadata_df)
articles_metadata_df['content'] = ["Example Content"] * len(articles_metadata_df)

# Création de la matrice TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(articles_metadata_df['title'] + " " + articles_metadata_df['content'])

# Réduction de dimensions avec TruncatedSVD
# Assurez-vous que n_components est inférieur ou égal au nombre de caractéristiques
n_components = min(100, tfidf_matrix.shape[1])
svd = TruncatedSVD(n_components=n_components)  # Réduire à 100 dimensions ou moins
reduced_tfidf_matrix = svd.fit_transform(tfidf_matrix)

# Convertir la matrice TF-IDF réduite en tenseur TensorFlow
reduced_tfidf_matrix_tensor = tf.convert_to_tensor(reduced_tfidf_matrix, dtype=tf.float32)

# Normaliser la matrice TF-IDF
normalized_tfidf_matrix = tf.nn.l2_normalize(reduced_tfidf_matrix_tensor, axis=1)

# Fonction pour calculer la similarité cosinus en utilisant des lots
def batch_cosine_similarity(matrix, batch_size=1000):
    num_rows = matrix.shape[0]
    cosine_sim_matrix = np.zeros((num_rows, num_rows), dtype=np.float32)

    for start_idx in range(0, num_rows, batch_size):
        end_idx = min(start_idx + batch_size, num_rows)
        batch = matrix[start_idx:end_idx]
        similarity = tf.matmul(batch, matrix, transpose_b=True).numpy()
        cosine_sim_matrix[start_idx:end_idx] = similarity

    return cosine_sim_matrix

# Calcul de la similarité cosinus en utilisant des lots
cosine_sim = batch_cosine_similarity(normalized_tfidf_matrix, batch_size=1000)

# Vérifions la matrice de similarité
print("Aperçu de la matrice de similarité:")
print(cosine_sim[:5, :5])  # Affichage d'un aperçu pour éviter trop de données

# Fonction de recommandation basée sur le contenu
def recommend_articles_content_based(article_id, num_recommendations=5):
    article_idx = articles_metadata_df[articles_metadata_df['article_id'] == article_id].index[0]
    similar_articles = list(enumerate(cosine_sim[article_idx]))
    similar_articles = sorted(similar_articles, key=lambda x: x[1], reverse=True)
    recommended_articles = [articles_metadata_df['article_id'].iloc[i[0]] for i in similar_articles[1:num_recommendations+1]]
    return recommended_articles

# Exemple de recommandation d'articles basée sur le contenu pour un article donné
article_id_example = articles_metadata_df['article_id'].iloc[0]
print(f"Recommandations basées sur le contenu pour l'article {article_id_example} : {recommend_articles_content_based(article_id_example)}")


## Fonction de recommandation basée sur le contenu

In [ ]:
# Vérifiez si TensorFlow utilise le GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Chargement des données
articles_metadata_df = pd.read_csv('../data/articles_metadata.csv')

# Ajouter des colonnes fictives 'title' et 'content' pour la démonstration
articles_metadata_df['title'] = ["Example Title"] * len(articles_metadata_df)
articles_metadata_df['content'] = ["Example Content"] * len(articles_metadata_df)

# Création de la matrice TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(articles_metadata_df['title'] + " " + articles_metadata_df['content'])

# Réduction de dimensions avec TruncatedSVD
# Assurez-vous que n_components est inférieur ou égal au nombre de caractéristiques
n_components = min(100, tfidf_matrix.shape[1])
svd = TruncatedSVD(n_components=n_components)  # Réduire à 100 dimensions ou moins
reduced_tfidf_matrix = svd.fit_transform(tfidf_matrix)

# Convertir la matrice TF-IDF réduite en tenseur TensorFlow
reduced_tfidf_matrix_tensor = tf.convert_to_tensor(reduced_tfidf_matrix, dtype=tf.float32)

# Normaliser la matrice TF-IDF
normalized_tfidf_matrix = tf.nn.l2_normalize(reduced_tfidf_matrix_tensor, axis=1)

# Fonction pour calculer la similarité cosinus en utilisant des lots
def batch_cosine_similarity(matrix, batch_size=1000):
    num_rows = matrix.shape[0]
    cosine_sim_matrix = np.zeros((num_rows, num_rows), dtype=np.float32)

    for start_idx in range(0, num_rows, batch_size):
        end_idx = min(start_idx + batch_size, num_rows)
        batch = matrix[start_idx:end_idx]
        similarity = tf.matmul(batch, matrix, transpose_b=True).numpy()
        cosine_sim_matrix[start_idx:end_idx] = similarity

    return cosine_sim_matrix

# Calcul de la similarité cosinus en utilisant des lots
cosine_sim = batch_cosine_similarity(normalized_tfidf_matrix, batch_size=1000)

# Vérifions la matrice de similarité
print("Aperçu de la matrice de similarité:")
print(cosine_sim[:5, :5])  # Affichage d'un aperçu pour éviter trop de données

# Fonction de recommandation basée sur le contenu avec gestion par lots
def recommend_articles_content_based(article_id, num_recommendations=5, batch_size=1000):
    article_idx = articles_metadata_df[articles_metadata_df['article_id'] == article_id].index[0]
    num_articles = cosine_sim.shape[0]

    # Calcul de la similarité par lots
    similar_articles = []
    for start_idx in range(0, num_articles, batch_size):
        end_idx = min(start_idx + batch_size, num_articles)
        batch_similarities = cosine_sim[article_idx, start_idx:end_idx]
        batch_indices = range(start_idx, end_idx)
        similar_articles.extend(zip(batch_indices, batch_similarities))

    similar_articles = sorted(similar_articles, key=lambda x: x[1], reverse=True)
    recommended_articles = [articles_metadata_df['article_id'].iloc[i[0]] for i in similar_articles[1:num_recommendations+1]]
    return recommended_articles

# Exemple de recommandation d'articles basée sur le contenu pour un article donné
article_id_example = articles_metadata_df['article_id'].iloc[0]
print(f"Recommandations basées sur le contenu pour l'article {article_id_example} : {recommend_articles_content_based(article_id_example)}")


## Recommandation hybride combinant filtrage collaboratif et basé sur le contenu

In [ ]:
# Vérifiez si TensorFlow utilise le GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Chargement des données
articles_metadata_df = pd.read_csv('../data/articles_metadata.csv')
clicks_sample_df = pd.read_csv('../data/clicks_sample.csv')

# Préparation des données pour le filtrage collaboratif
article_user_matrix = clicks_sample_df.pivot_table(index='user_id', columns='click_article_id', values='session_id', aggfunc='count').fillna(0)

# Calcul de la similarité cosinus pour le filtrage collaboratif
article_similarity = cosine_similarity(article_user_matrix.T)

# Fonction de recommandation basée sur le filtrage collaboratif
def recommend_articles_article_based(article_id, num_recommendations=5):
    article_idx = article_user_matrix.columns.get_loc(article_id)
    similar_articles = list(enumerate(article_similarity[article_idx]))
    similar_articles = sorted(similar_articles, key=lambda x: x[1], reverse=True)
    recommended_articles = [article_user_matrix.columns[i[0]] for i in similar_articles[1:num_recommendations+1]]
    return recommended_articles

# Ajouter des colonnes fictives 'title' et 'content' pour la démonstration
articles_metadata_df['title'] = ["Example Title"] * len(articles_metadata_df)
articles_metadata_df['content'] = ["Example Content"] * len(articles_metadata_df)

# Création de la matrice TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(articles_metadata_df['title'] + " " + articles_metadata_df['content'])

# Réduction de dimensions avec TruncatedSVD
n_components = min(100, tfidf_matrix.shape[1])
svd = TruncatedSVD(n_components=n_components)
reduced_tfidf_matrix = svd.fit_transform(tfidf_matrix)

# Convertir la matrice TF-IDF réduite en tenseur TensorFlow
reduced_tfidf_matrix_tensor = tf.convert_to_tensor(reduced_tfidf_matrix, dtype=tf.float32)

# Normaliser la matrice TF-IDF
normalized_tfidf_matrix = tf.nn.l2_normalize(reduced_tfidf_matrix_tensor, axis=1)

# Fonction pour calculer la similarité cosinus en utilisant des lots
def batch_cosine_similarity(matrix, batch_size=1000):
    num_rows = matrix.shape[0]
    cosine_sim_matrix = np.zeros((num_rows, num_rows), dtype=np.float32)

    for start_idx in range(0, num_rows, batch_size):
        end_idx = min(start_idx + batch_size, num_rows)
        batch = matrix[start_idx:end_idx]
        similarity = tf.matmul(batch, matrix, transpose_b=True).numpy()
        cosine_sim_matrix[start_idx:end_idx] = similarity

    return cosine_sim_matrix

# Calcul de la similarité cosinus en utilisant des lots
cosine_sim = batch_cosine_similarity(normalized_tfidf_matrix, batch_size=1000)

# Fonction de recommandation basée sur le contenu
def recommend_articles_content_based(article_id, num_recommendations=5):
    article_idx = articles_metadata_df[articles_metadata_df['article_id'] == article_id].index[0]
    similar_articles = list(enumerate(cosine_sim[article_idx]))
    similar_articles = sorted(similar_articles, key=lambda x: x[1], reverse=True)
    recommended_articles = [articles_metadata_df['article_id'].iloc[i[0]] for i in similar_articles[1:num_recommendations+1]]
    return recommended_articles

# Recommandation hybride combinant filtrage collaboratif et basé sur le contenu
def hybrid_recommendation(user_id, num_recommendations=5):
    # Recommandations basées sur le filtrage collaboratif
    user_clicks = article_user_matrix.loc[user_id]
    user_clicks = user_clicks[user_clicks > 0]
    collaborative_recommendations = []
    for article_id in user_clicks.index:
        collaborative_recommendations.extend(recommend_articles_article_based(article_id, num_recommendations))
    collaborative_recommendations = list(set(collaborative_recommendations))
    
    # Recommandations basées sur le contenu
    content_based_recommendations = []
    for article_id in user_clicks.index:
        content_based_recommendations.extend(recommend_articles_content_based(article_id, num_recommendations))
    content_based_recommendations = list(set(content_based_recommendations))
    
    # Combiner les recommandations
    combined_recommendations = list(set(collaborative_recommendations + content_based_recommendations))
    return combined_recommendations[:num_recommendations]

# Exemple de recommandation hybride pour un utilisateur donné
user_id_example = article_user_matrix.index[0]
print(f"Recommandations hybrides pour l'utilisateur {user_id_example} : {hybrid_recommendation(user_id_example)}")


## Sauvegarde des données prétraitées

Nous allons sauvegarder les données prétraitées pour une utilisation ultérieure dans l'entraînement du modèle.

In [ ]:
import joblib
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from flask import Flask, request, jsonify
import joblib
import numpy as np
import pandas as pd

# Chemins relatifs pour sauvegarder les modèles et les données
article_user_matrix_path = 'models/article_user_matrix.pkl'
article_similarity_path = 'models/article_similarity.pkl'
tfidf_vectorizer_path = 'models/tfidf_vectorizer.pkl'
tfidf_matrix_path = 'models/tfidf_matrix.pkl'
cosine_sim_path = 'models/cosine_sim.pkl'

# Créer le répertoire 'models' s'il n'existe pas
import os
if not os.path.exists('models'):
    os.makedirs('models')

# Enregistrer et suivre les modèles
joblib.dump(article_user_matrix, article_user_matrix_path)
joblib.dump(article_similarity, article_similarity_path)
joblib.dump(tfidf_vectorizer, tfidf_vectorizer_path)
joblib.dump(tfidf_matrix, tfidf_matrix_path)
joblib.dump(cosine_sim, cosine_sim_path)

print("Modèles et données préparées sauvegardés.")

## Déploiement Serverless avec Azure Functions
Api local de test

In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np
import pandas as pd

app = Flask(__name__)

# Chemins relatifs des fichiers sauvegardés
article_user_matrix_path = 'models/article_user_matrix.pkl'
article_similarity_path = 'models/article_similarity.pkl'
tfidf_vectorizer_path = 'models/tfidf_vectorizer.pkl'
tfidf_matrix_path = 'models/tfidf_matrix.pkl'
cosine_sim_path = 'models/cosine_sim.pkl'

# Charger les fichiers sauvegardés
article_user_matrix = joblib.load(article_user_matrix_path)
article_similarity = joblib.load(article_similarity_path)
tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)
tfidf_matrix = joblib.load(tfidf_matrix_path)
cosine_sim = joblib.load(cosine_sim_path)

# Fonction de recommandation collaborative
def collaborative_recommendation(user_id, num_recommendations=5):
    if user_id in article_user_matrix.index:
        user_clicks = article_user_matrix.loc[user_id]
        user_clicks = user_clicks[user_clicks > 0]
        collaborative_recommendations = []
        for article_id in user_clicks.index:
            similar_articles = np.argsort(article_similarity[article_id])[::-1]
            collaborative_recommendations.extend(similar_articles[:num_recommendations])
        collaborative_recommendations = list(set(collaborative_recommendations))
        return collaborative_recommendations[:num_recommendations]
    else:
        return []

# Fonction de recommandation basée sur le contenu
def content_based_recommendation(user_id, num_recommendations=5):
    if user_id in article_user_matrix.index:
        user_clicks = article_user_matrix.loc[user_id]
        user_clicks = user_clicks[user_clicks > 0]
        content_based_recommendations = []
        for article_id in user_clicks.index:
            similar_articles = np.argsort(cosine_sim[article_id])[::-1]
            content_based_recommendations.extend(similar_articles[:num_recommendations])
        content_based_recommendations = list(set(content_based_recommendations))
        return content_based_recommendations[:num_recommendations]
    else:
        return []

# Fonction de recommandation hybride
def hybrid_recommendation(user_id, num_recommendations=5):
    collaborative_recommendations = collaborative_recommendation(user_id, num_recommendations)
    content_based_recommendations = content_based_recommendation(user_id, num_recommendations)
    combined_recommendations = list(set(collaborative_recommendations + content_based_recommendations))
    return combined_recommendations[:num_recommendations]

@app.route('/recommend/collaborative', methods=['GET'])
def recommend_collaborative():
    user_id = int(request.args.get('user_id'))
    recommendations = collaborative_recommendation(user_id)
    return jsonify(recommendations)

@app.route('/recommend/content', methods=['GET'])
def recommend_content():
    user_id = int(request.args.get('user_id'))
    recommendations = content_based_recommendation(user_id)
    return jsonify(recommendations)

@app.route('/recommend/hybrid', methods=['GET'])
def recommend_hybrid():
    user_id = int(request.args.get('user_id'))
    recommendations = hybrid_recommendation(user_id)
    return jsonify(recommendations)

if __name__ == '__main__':
    app.run(debug=True)


## Web app


In [ ]:
from flask import Flask, request, jsonify, render_template
import joblib
import numpy as np
import pandas as pd

app = Flask(__name__, template_folder='../app/templates')

# Chemins relatifs des fichiers sauvegardés
article_user_matrix_path = 'models/article_user_matrix.pkl'
article_similarity_path = 'models/article_similarity.pkl'
tfidf_vectorizer_path = 'models/tfidf_vectorizer.pkl'
tfidf_matrix_path = 'models/tfidf_matrix.pkl'
cosine_sim_path = 'models/cosine_sim.pkl'

# Charger les fichiers sauvegardés
article_user_matrix = joblib.load(article_user_matrix_path)
article_similarity = joblib.load(article_similarity_path)
tfidf_vectorizer = joblib.load(tfidf_vectorizer_path)
tfidf_matrix = joblib.load(tfidf_matrix_path)
cosine_sim = joblib.load(cosine_sim_path)

# Fonction de recommandation collaborative
def collaborative_recommendation(user_id, num_recommendations=5):
    if user_id in article_user_matrix.index:
        user_clicks = article_user_matrix.loc[user_id]
        user_clicks = user_clicks[user_clicks > 0]
        collaborative_recommendations = []
        for article_id in user_clicks.index:
            similar_articles = np.argsort(article_similarity[article_id])[::-1]
            collaborative_recommendations.extend(similar_articles[:num_recommendations])
        collaborative_recommendations = list(set(collaborative_recommendations))
        return collaborative_recommendations[:num_recommendations]
    else:
        return []

# Fonction de recommandation basée sur le contenu
def content_based_recommendation(user_id, num_recommendations=5):
    if user_id in article_user_matrix.index:
        user_clicks = article_user_matrix.loc[user_id]
        user_clicks = user_clicks[user_clicks > 0]
        content_based_recommendations = []
        for article_id in user_clicks.index:
            similar_articles = np.argsort(cosine_sim[article_id])[::-1]
            content_based_recommendations.extend(similar_articles[:num_recommendations])
        content_based_recommendations = list(set(content_based_recommendations))
        return content_based_recommendations[:num_recommendations]
    else:
        return []

# Fonction de recommandation hybride
def hybrid_recommendation(user_id, num_recommendations=5):
    collaborative_recommendations = collaborative_recommendation(user_id, num_recommendations)
    content_based_recommendations = content_based_recommendation(user_id, num_recommendations)
    combined_recommendations = list(set(collaborative_recommendations + content_based_recommendations))
    return combined_recommendations[:num_recommendations]

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/recommend/collaborative', methods=['GET'])
def recommend_collaborative():
    user_id = int(request.args.get('user_id'))
    recommendations = collaborative_recommendation(user_id)
    return jsonify(recommendations)

@app.route('/recommend/content', methods=['GET'])
def recommend_content():
    user_id = int(request.args.get('user_id'))
    recommendations = content_based_recommendation(user_id)
    return jsonify(recommendations)

@app.route('/recommend/hybrid', methods=['GET'])
def recommend_hybrid():
    user_id = int(request.args.get('user_id'))
    recommendations = hybrid_recommendation(user_id)
    return jsonify(recommendations)

if __name__ == '__main__':
    app.run(debug=True)